Modified code originally provided by Zona.

In [ ]:
import torch
from torch.autograd import Variable as V
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms as trn
import numpy as np
import cv2
from PIL import Image
import pickle

In [ ]:
def returnTF():
# process image before being input into CNN model
  tf = trn.Compose([
    trn.Resize((256,256)),
    trn.CenterCrop(224),
    trn.ToTensor(),
    trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  return tf

def load_model():

  #load pretrained places CNN model provided by Zona
  with open('cnn_model.pkl', 'rb') as f:
    cnn_model = pickle.load(f)

    #remove classification and avgpool layers from model
    model = nn.Sequential(*list(cnn_model.children())[:-2])

  return model

def get_cnn_features(img):
  '''
  Retrieves features from CNN model from input image
  '''
  model = load_model()

  # load and apply the transformer
  tf = returnTF()
  input_img = V(tf(img).unsqueeze(0))

  #classification layer removed so output of model is output of last layer of CNN
  features_blobs = model(input_img).cpu().detach().numpy().squeeze(0)
  features = cv2.resize(features_blobs, (18,18), interpolation=cv2.INTER_AREA).flatten()[:2048] #resize to 2048 elements same as provided dataset
  features[features == 0] = -1 #replace zeros with -1 to match data cleaning treatment for dataset

  return features